In [1]:
import numpy as np

In [2]:
class Tensor(object):
    """docstring for tensor"""
    def __init__(self, initial_value,op,graph):
        self.initial_value = initial_value
        self.graph = graph
        self.op = op

    def __add__(self, other): # change the + operateor's methods, other is another self 
        return self.graph.add(self, other)

In [3]:
class BaseOp(object):
    """docstring for BaseOp"""
    def __init__(self, inputs,graph):
        self.inputs = [graph.convert(input_) for input_ in inputs]
        self.output = graph.tensor(op=self)
        self.graph  = graph

    def compute(self,sess,*args):
        raise NotImplementedError()
    def gradient(self,grad):
        raise NotImplementedError()
'''
class myParent( object ):
    def __init__( self, customParam ):
        self.parentNumber = 5
        self.customParam = customParam

class Child( myParent ):
    def __init__( self, customParam ):
        myParent.__init__( self, customParam )
        self.childNumber = 4        
'''        
        
class AddOp(BaseOp):
    def __init__(self, inputs,graph):
        BaseOp.__init__(self, inputs,graph)
        self.output = graph.convert(   self.compute(inputs[0],inputs[1])   )
        #self.output = graph.convert(1000000000000000000000000000000)
    def compute(self,a=0,b=0):
        #a = self.inputs[0].initial_value
        #b = self.inputs[1].initial_value
        return a.initial_value + b.initial_value
    def gradient(self,grad):
        return [grad,grad]

In [4]:
class Graph(object):
	"""docstring for Graph"""
	def tensor(self,initial_value=None,op=None):
		return Tensor(initial_value=initial_value,op=op,graph=self)

	def convert(self,value):
		if isinstance(value,Tensor):
			return value
		return self.tensor(initial_value=value)

	def gradients(self,y,x_s):

		'''
		>>>queue.append(([1,1],1))
		>>>queue.pop(0)
		([1, 1], 1)



    	loss_op = graph.mean(graph.square(graph.transpose(y) - activations1))
    	parameters = [weights0, biases0, weights1, biases1]

    	gradients = graph.gradients(loss_op, parameters)
		'''
		queue = []
		queue.append((y,1))

		grads = {}

		while len(queue)>0:
			y,grad_y = queue.pop(0)
			grad_y = self.convert(grad_y) #convert grad_y to a tensor

			gradients = y.op.gradient(grad_y)
		return gradients

	def add(self,a,b):
		op = AddOp([a,b],graph=self)
		return op.output

In [5]:
test_graph = Graph()
test_X = test_graph.tensor([1,'2','a']) 
test_Y = test_graph.tensor([3,4,'b'])

In [6]:
print(test_X)
print(test_Y)
print(test_X.initial_value)
print(test_Y.initial_value)
print(test_X.op)
print(test_Y.op)

[1, '2', 'a']
[3, 4, 'b']
None
None


In [7]:
test_Z = test_X + test_Y + test_X
print(test_Z.initial_value)

[1, '2', 'a', 3, 4, 'b', 1, '2', 'a']
